In [9]:
%pip install pandas
%pip install numpy
%pip install scipy

import pandas as pd
import sys
import os
import string
import numpy as np
import scipy.stats as stats
import re

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):    
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [11]:
def calc_mean(list): 
   
    mean = np.mean(list)  
    return mean /1000 
    # std = np.std(list)
    # normal_list = []
    # nz=[]
    # for l in list:
    #     if std == 0:
    #         break
    #     z = (l-mean)/std
    #     if z <=0.5:
    #         normal_list.append(l)
    #     nz.append(z)
    # if len(normal_list) == 0:
    #     avg_value = min(list)         
    # else:
    #     avg_value = np.mean(normal_list)
    # return avg_value/1000  

In [12]:
def merge_results(paths):
    dfs = []
    base_index = 0
    for path in paths:        
        data = pd.read_csv( path, sep=',')
        df_index = [ *range(base_index, base_index + len(data), 1)]
        base_index += len(data)
        data.index=df_index
        dfs.append(data)


    result =  pd.concat(dfs)
    return result


def calc_read_write(header,baselines, languages, tasksets, platforms,seq_rands, nrows, df_run, df_load_to_memory, execution, rate):
   
    df_final = pd.DataFrame(columns = header)    
    for baseline in baselines:
        for language in languages:
            for taskset in tasksets:
                for platform in platforms:
                    for seq_rand in seq_rands:
                        for nrow in nrows:                                                                      
                            df_total =  df_run.loc[(df_run['baseline']==baseline) &
                                                   (df_run["language"]==language) &
                                                   (df_run['taskset']==taskset) &
                                                   (df_run['execution']=='Total') &
                                                   (df_run['platform']==platform) &
                                                   (df_run['seq_rand']==seq_rand) &
                                                   (df_run['nrow']==nrow)
                                                   ]

                            list_total = np.array(df_total['time'].tolist())  
                            df_cpu_io =  df_run.loc[(df_run['baseline']==baseline) &
                                                    (df_run["language"]==language) &
                                                    (df_run['taskset']==taskset) &
                                                    (df_run['execution']==execution) &
                                                    (df_run['platform']==platform) &
                                                    (df_run['seq_rand']==seq_rand) &
                                                    (df_run['nrow']==nrow)]

                            list_cpu_io = np.array(df_cpu_io['time'].tolist())  

                            df_load =  df_load_to_memory.loc[(df_load_to_memory["language"]==language) &
                                                             (df_load_to_memory['taskset']==taskset) & 
                                                             (df_load_to_memory['platform']==platform) &
                                                             (df_load_to_memory['nrow']==nrow)]

                            list_load = np.array(df_load['time'].tolist())  

                            if len(list_total) > 0:
                                time_total = calc_mean(list=list_total)                                               
                                base = baseline.replace("FlatBuffersJava", "FlatBufJava")                                    
                                if execution == "CPU" :  
                                    time_load = calc_mean(list=list_load)                                                      
                                    total = time_total - time_load 
                                     
                                elif execution == "IO":                                 
                                        total = time_total

                                result_total = [base,language,taskset,'Total',platform,seq_rand,nrow,f'{total/rate:.2f}']    
                                df_final.loc[len(df_final)] = result_total    
                                 

                            if len(list_cpu_io) > 0:
                                    time_cpu_io = calc_mean(list=list_cpu_io)                                                                     
                                    base = baseline.replace("FlatBuffersJava", "FlatBufJava")                                    
                                    if execution == "CPU" :
                                        io  = time_total - time_cpu_io
                                        cpu  = time_cpu_io - time_load
                                     
                                    elif execution == "IO":                                           
                                        cpu = time_total - time_cpu_io       
                                        io = time_cpu_io
                                    
                                    result_cpu = [base,language,taskset,'CPU',platform,seq_rand,nrow,f'{cpu/rate:.2f}']
                                    result_io = [base,language,taskset,"IO",platform,seq_rand,nrow,f'{io/rate:.2f}']    
                                    
                                    df_final.loc[len(df_final)] = result_cpu
                                    df_final.loc[len(df_final)] = result_io



  
    return df_final  

    

In [13]:
def calc_network(header,baselines, languages, df_run):
       
    df_final = pd.DataFrame(columns = header)    
    for baseline in baselines:
        for language in languages:
            df_total =  df_run.loc[(df_run['baseline']==baseline) &
                                   (df_run["language"]==language) &
                                   (df_run['execution']=='Total')
                                    ]
            list_total = np.array(df_total['time'].tolist())  

            df_io =  df_run.loc[(df_run['baseline']==baseline) &
                                   (df_run["language"]==language) &
                                   (df_run['execution']=='IO')
                                ]
            list_io = np.array(df_io['time'].tolist())  
            if len(list_total) > 0:
                 time_total = calc_mean(list=list_total) / 60    
                 time_io = calc_mean(list=list_io) / 60
                 time_cpu = time_total - time_io
                 base = baseline.replace("FlatBuffersJava", "FlatBufJava")
                 result_total = [base,language,'Total',f'{time_total:.2f}']
                 result_cpu = [base,language,'CPU',f'{time_cpu:.2f}']
                 result_io = [base,language,"IO",f'{time_io:.2f}']    
               
                 df_final.loc[len(df_final)] = result_total
                 df_final.loc[len(df_final)] = result_cpu
                 df_final.loc[len(df_final)] = result_io  
    return df_final 

In [14]:
def calc_externalsort(header,baselines,tasksets, languages, df_run):
   # baseline,language,taskset,execution,chunk_size,nrow,time
   
    df_final = pd.DataFrame(columns = header)    
    for baseline in baselines:
        for language in languages:
            for taskset in tasksets:
                df_total =  df_run.loc[(df_run['baseline']==baseline) &
                                       (df_run["language"]==language) &
                                       (df_run["taskset"]==taskset) &
                                       (df_run['execution']=='Total')
                                        ]
                list_total = np.array(df_total['time'].tolist())  

                df_io =  df_run.loc[(df_run['baseline']==baseline) &
                                    (df_run["language"]==language) &
                                    (df_run["taskset"]==taskset) &
                                    (df_run['execution']=='IO')
                                    ]
                list_io = np.array(df_io['time'].tolist())  
                if len(list_total) > 0:
                    time_total = calc_mean(list=list_total) / 60    
                    time_io = calc_mean(list=list_io) / 60
                    time_cpu = time_total - time_io
                    base = baseline.replace("FlatBuffersJava", "FlatBufJava")
                    result_total = [base,language,taskset,'Total',1000000,15000000,f'{time_total:.2f}']
                    result_cpu = [base,language,taskset,'CPU',1000000,15000000,f'{time_cpu:.2f}']
                    result_io = [base,language,taskset,"IO",1000000,15000000,f'{time_io:.2f}']    
                
                    df_final.loc[len(df_final)] = result_total
                    df_final.loc[len(df_final)] = result_cpu
                    df_final.loc[len(df_final)] = result_io  
    return df_final 

In [15]:
def results():
    root_path = '../archive/results'    

    write_results = [f'{root_path}/Java_Experiment1_Write_times.dat',
                     f'{root_path}/CPP_Experiment1_Write_times.dat',
                     f'{root_path}/Rust_Experiment1_Write_times.dat']

    read_results = [f'{root_path}/Java_Experiment2_Read_times.dat',
                    f'{root_path}/CPP_Experiment2_Read_times.dat',
                    f'{root_path}/Rust_Experiment2_Read_times.dat']

    load_to_memory_results = [f'{root_path}/Java_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/CPP_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/Rust_Experiment1_LoadToMemory_times.dat']

    network_results = [f'{root_path}/CPP_Experiment4_Network_ExternalSort_times.dat',
                       f'{root_path}/Java_Experiment4_Network_ExternalSort_times.dat',
                       f'{root_path}/Rust_Experiment4_Network_ExternalSort_times.dat'] 
                       
    # externalsort_results = [f'{root_path}/CPP_Experiment3b_ExternalSort_times.dat',
    #                         f'{root_path}/Java_Experiment3b_ExternalSort_times.dat',
    #                         f'{root_path}/Rust_Experiment3b_ExternalSort_times.dat']    
    externalsort_results = [f'{root_path}/Experiment3b_ExternalSort_times.dat']                   
   
    read_write_header = ["baseline","language","taskset","execution","platform","seq_rand","nrow","time"]      
    load_to_memory_header = ["language","platform","taskset","nrow","time"]
    network_header = ["baseline","language","execution","time"]  
    externalsort_header = ["baseline","language","taskset","execution","chunk_size","nrow","time"]  

    baselines = ["DefaultJava","Json+GzipJava","BsonJava","ProtoBufJava","KryoJava","ByteBufferJava","JsonJava","FlatBuffersJava",
                "HandcodedCPP","inPlaceCPP","BoostCPP", "BoostBinaryCPP","ProtoBufCPP","BsonCPP","FlatBufCPP",
                "JsonRust","BincodeRust","MessagePackRust","BsonRust","FlexBufRust"]    

     
    
    nrows = [10000000,8000000,6000000,4000000,2000000]

    executions = ["CPU", "IO", "Total"]
    platforms  = ["Parallel","Single"] 
    seq_rands  = ["Sequential", "Random"]
    languages  = ["Java", "CPP","Rust"]
    tasksets   = [True, False]

    df_write = merge_results(write_results)    
    df_read = merge_results(read_results)
    df_load = merge_results(load_to_memory_results)   
    df_network = merge_results(network_results)   
    df_externalsort = merge_results(externalsort_results)   
   
    df_write_single_final = calc_read_write(header=read_write_header, baselines=baselines, languages=languages, tasksets=tasksets, platforms=["Single"],
                                       seq_rands= seq_rands, nrows=[10000000], df_run=df_write, df_load_to_memory= df_load, execution="CPU", rate=60)

    df_write_parallel_final = calc_read_write(header=read_write_header, baselines=baselines, languages=languages, tasksets=tasksets, platforms=["Parallel"],seq_rands= seq_rands, nrows=[10000000], df_run=df_write, df_load_to_memory= df_load, execution="CPU", rate=1)                                  


    df_read_single_final = calc_read_write(header=read_write_header, baselines=baselines, languages=languages, tasksets=tasksets, platforms=["Single"],
                                       seq_rands= seq_rands, nrows=nrows, df_run=df_read, df_load_to_memory= df_load, execution="IO", rate=60)   

    df_read_parallel_final = calc_read_write(header=read_write_header, baselines=baselines, languages=languages, tasksets=tasksets, platforms=["Parallel"], seq_rands= seq_rands, nrows=nrows, df_run=df_read, df_load_to_memory= df_load, execution="IO", rate=60) 

    df_read_single_10m_final = calc_read_write(header=read_write_header, baselines=baselines, languages=languages, tasksets=tasksets, platforms=["Single"],seq_rands= seq_rands, nrows=[10000000], df_run=df_read, df_load_to_memory= df_load, execution="IO", rate=60) 

    df_read_parallel_10m_final = calc_read_write(header=read_write_header, baselines=baselines, languages=languages, tasksets=tasksets, platforms=["Parallel"], seq_rands= seq_rands, nrows=[10000000], df_run=df_read, df_load_to_memory= df_load, execution="IO", rate=60)                                                                       

                         

    # df_network_final = calc_network(header=network_header, baselines=baselines, languages=languages, df_run=df_network)

    df_externalsort_final = calc_externalsort(header=externalsort_header,tasksets=tasksets, baselines=baselines, languages=languages, df_run=df_externalsort)
        
               
    # df_write_single_final.to_csv("results/Exp1_write_single.dat", index=False)  
    # df_write_parallel_final.to_csv("results/Exp1_write_parallel.dat", index=False)
    # df_read_single_final.to_csv("results/Exp2_read_single.dat", index=False)
    # df_read_parallel_final.to_csv("results/Exp2_read_parallel.dat", index=False)
    # df_read_single_10m_final.to_csv("results/Exp2_read_single_10m.dat", index=False)
    # df_read_parallel_10m_final.to_csv("results/Exp2_read_parallel_10m.dat", index=False)
    # df_network_final.to_csv("results/Exp5_network.dat", index=False)      
    df_externalsort_final.to_csv("results/Exp4_externalsort.dat", index=False)            
  


   

In [16]:
results()